# 論文 13：注意力就是你所需要的（Attention Is All You Need）
## Vaswani 等人（2017）

### Transformer：純注意力架構

革命性的架構，用自注意力取代了 RNN，實現了現代大型語言模型（LLM）。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 縮放點積注意力（Scaled Dot-Product Attention）

基本構建塊：
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

In [ ]:
def softmax(x, axis=-1):
    """數值穩定的 softmax"""
    x_max = np.max(x, axis=axis, keepdims=True)
    exp_x = np.exp(x - x_max)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    縮放點積注意力
    
    Q：查詢（Queries）(seq_len_q, d_k)
    K：鍵（Keys）(seq_len_k, d_k)
    V：值（Values）(seq_len_v, d_v)
    mask：可選的遮罩 (seq_len_q, seq_len_k)
    """
    d_k = Q.shape[-1]
    
    # 計算注意力分數
    scores = np.dot(Q, K.T) / np.sqrt(d_k)
    
    # 如果提供了遮罩，則應用遮罩（用於因果關係或填充）
    if mask is not None:
        scores = scores + (mask * -1e9)
    
    # Softmax 獲得注意力權重
    attention_weights = softmax(scores, axis=-1)
    
    # 值的加權和
    output = np.dot(attention_weights, V)
    
    return output, attention_weights

# 測試縮放點積注意力
seq_len = 5
d_model = 8

Q = np.random.randn(seq_len, d_model)
K = np.random.randn(seq_len, d_model)
V = np.random.randn(seq_len, d_model)

output, attn_weights = scaled_dot_product_attention(Q, K, V)

print(f"注意力輸出形狀：{output.shape}")
print(f"注意力權重形狀：{attn_weights.shape}")
print(f"注意力權重總和（應為 1）：{attn_weights.sum(axis=1)}")

# 視覺化注意力模式
plt.figure(figsize=(8, 6))
plt.imshow(attn_weights, cmap='viridis', aspect='auto')
plt.colorbar(label='注意力權重')
plt.xlabel('鍵位置')
plt.ylabel('查詢位置')
plt.title('注意力權重矩陣')
plt.show()

## 多頭注意力（Multi-Head Attention）

多個注意力「頭」關注輸入的不同方面：
$$\text{MultiHead}(Q,K,V) = \text{Concat}(head_1, ..., head_h)W^O$$

In [ ]:
class MultiHeadAttention:
    def __init__(self, d_model, num_heads):
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # 所有頭的 Q、K、V 線性投影（並行化）
        self.W_q = np.random.randn(d_model, d_model) * 0.1
        self.W_k = np.random.randn(d_model, d_model) * 0.1
        self.W_v = np.random.randn(d_model, d_model) * 0.1
        
        # 輸出投影
        self.W_o = np.random.randn(d_model, d_model) * 0.1
    
    def split_heads(self, x):
        """分成多個頭：(seq_len, d_model) -> (num_heads, seq_len, d_k)"""
        seq_len = x.shape[0]
        x = x.reshape(seq_len, self.num_heads, self.d_k)
        return x.transpose(1, 0, 2)
    
    def combine_heads(self, x):
        """合併頭：(num_heads, seq_len, d_k) -> (seq_len, d_model)"""
        seq_len = x.shape[1]
        x = x.transpose(1, 0, 2)
        return x.reshape(seq_len, self.d_model)
    
    def forward(self, Q, K, V, mask=None):
        """
        多頭注意力前向傳遞
        
        Q, K, V：(seq_len, d_model)
        """
        # 線性投影
        Q = np.dot(Q, self.W_q.T)
        K = np.dot(K, self.W_k.T)
        V = np.dot(V, self.W_v.T)
        
        # 分成多個頭
        Q = self.split_heads(Q)  # (num_heads, seq_len, d_k)
        K = self.split_heads(K)
        V = self.split_heads(V)
        
        # 對每個頭應用注意力
        head_outputs = []
        self.attention_weights = []
        
        for i in range(self.num_heads):
            head_out, head_attn = scaled_dot_product_attention(
                Q[i], K[i], V[i], mask
            )
            head_outputs.append(head_out)
            self.attention_weights.append(head_attn)
        
        # 堆疊頭
        heads = np.stack(head_outputs, axis=0)  # (num_heads, seq_len, d_k)
        
        # 合併頭
        combined = self.combine_heads(heads)  # (seq_len, d_model)
        
        # 最終線性投影
        output = np.dot(combined, self.W_o.T)
        
        return output

# 測試多頭注意力
d_model = 64
num_heads = 8
seq_len = 10

mha = MultiHeadAttention(d_model, num_heads)

X = np.random.randn(seq_len, d_model)
output = mha.forward(X, X, X)  # 自注意力

print(f"\n多頭注意力：")
print(f"輸入形狀：{X.shape}")
print(f"輸出形狀：{output.shape}")
print(f"頭數：{num_heads}")
print(f"每個頭的維度：{mha.d_k}")

## 位置編碼（Positional Encoding）

由於 Transformer 沒有遞迴，我們添加位置資訊：
$$PE_{(pos, 2i)} = \sin(pos / 10000^{2i/d_{model}})$$
$$PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i/d_{model}})$$

In [ ]:
def positional_encoding(seq_len, d_model):
    """
    創建正弦位置編碼
    """
    pe = np.zeros((seq_len, d_model))
    
    position = np.arange(0, seq_len)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    
    # 對偶數索引應用 sin
    pe[:, 0::2] = np.sin(position * div_term)
    
    # 對奇數索引應用 cos
    pe[:, 1::2] = np.cos(position * div_term)
    
    return pe

# 生成位置編碼
seq_len = 50
d_model = 64
pe = positional_encoding(seq_len, d_model)

# 視覺化位置編碼
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.imshow(pe.T, cmap='RdBu', aspect='auto')
plt.colorbar(label='編碼值')
plt.xlabel('位置')
plt.ylabel('維度')
plt.title('位置編碼（所有維度）')

plt.subplot(2, 1, 2)
# 繪製前幾個維度
for i in [0, 1, 2, 3, 10, 20]:
    plt.plot(pe[:, i], label=f'維度 {i}')
plt.xlabel('位置')
plt.ylabel('編碼值')
plt.title('位置編碼（選定維度）')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"位置編碼形狀：{pe.shape}")
print(f"不同頻率在不同尺度上編碼位置")

## 前饋網路（Feed-Forward Network）

獨立應用於每個位置：
$$FFN(x) = \max(0, xW_1 + b_1)W_2 + b_2$$

In [ ]:
class FeedForward:
    def __init__(self, d_model, d_ff):
        self.W1 = np.random.randn(d_model, d_ff) * 0.1
        self.b1 = np.zeros(d_ff)
        self.W2 = np.random.randn(d_ff, d_model) * 0.1
        self.b2 = np.zeros(d_model)
    
    def forward(self, x):
        # 帶 ReLU 的第一層
        hidden = np.maximum(0, np.dot(x, self.W1) + self.b1)
        
        # 第二層
        output = np.dot(hidden, self.W2) + self.b2
        
        return output

# 測試前饋網路
d_model = 64
d_ff = 256  # 通常是 4 倍大

ff = FeedForward(d_model, d_ff)
x = np.random.randn(10, d_model)
output = ff.forward(x)

print(f"\n前饋網路：")
print(f"輸入：{x.shape}")
print(f"隱藏層：({x.shape[0]}, {d_ff})")
print(f"輸出：{output.shape}")

## 層正規化（Layer Normalization）

跨特徵正規化（不像 BatchNorm 那樣跨批次）

In [ ]:
class LayerNorm:
    def __init__(self, d_model, eps=1e-6):
        self.gamma = np.ones(d_model)
        self.beta = np.zeros(d_model)
        self.eps = eps
    
    def forward(self, x):
        mean = x.mean(axis=-1, keepdims=True)
        std = x.std(axis=-1, keepdims=True)
        
        normalized = (x - mean) / (std + self.eps)
        output = self.gamma * normalized + self.beta
        
        return output

ln = LayerNorm(d_model)
x = np.random.randn(10, d_model) * 3 + 5  # 未正規化的
normalized = ln.forward(x)

print(f"\n層正規化：")
print(f"輸入均值：{x.mean():.4f}，標準差：{x.std():.4f}")
print(f"輸出均值：{normalized.mean():.4f}，標準差：{normalized.std():.4f}")

## 完整的 Transformer 區塊

In [ ]:
class TransformerBlock:
    def __init__(self, d_model, num_heads, d_ff):
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = LayerNorm(d_model)
        self.ff = FeedForward(d_model, d_ff)
        self.norm2 = LayerNorm(d_model)
    
    def forward(self, x, mask=None):
        # 帶殘差連接的多頭注意力
        attn_output = self.attention.forward(x, x, x, mask)
        x = self.norm1.forward(x + attn_output)
        
        # 帶殘差連接的前饋網路
        ff_output = self.ff.forward(x)
        x = self.norm2.forward(x + ff_output)
        
        return x

# 測試 Transformer 區塊
block = TransformerBlock(d_model=64, num_heads=8, d_ff=256)
x = np.random.randn(10, 64)
output = block.forward(x)

print(f"\nTransformer 區塊：")
print(f"輸入形狀：{x.shape}")
print(f"輸出形狀：{output.shape}")
print(f"\n區塊包含：")
print(f"  1. 多頭自注意力")
print(f"  2. 層正規化")
print(f"  3. 前饋網路")
print(f"  4. 殘差連接")

## 視覺化多頭注意力模式

In [ ]:
# 創建具有可解釋輸入的注意力
seq_len = 8
d_model = 64
num_heads = 4

mha = MultiHeadAttention(d_model, num_heads)
X = np.random.randn(seq_len, d_model)
output = mha.forward(X, X, X)

# 繪製每個頭的注意力模式
fig, axes = plt.subplots(1, num_heads, figsize=(16, 4))

for i, ax in enumerate(axes):
    attn = mha.attention_weights[i]
    im = ax.imshow(attn, cmap='viridis', aspect='auto', vmin=0, vmax=1)
    ax.set_title(f'頭 {i+1}')
    ax.set_xlabel('鍵')
    ax.set_ylabel('查詢')
    
plt.colorbar(im, ax=axes, label='注意力權重', fraction=0.046, pad=0.04)
plt.suptitle('多頭注意力模式', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()

print("\n每個頭學習關注不同的模式！")
print("不同的頭捕捉資料中不同的關係。")

## 自迴歸模型的因果（遮罩）自注意力

In [ ]:
def create_causal_mask(seq_len):
    """創建遮罩以防止關注未來位置"""
    mask = np.triu(np.ones((seq_len, seq_len)), k=1)
    return mask

# 測試因果注意力
seq_len = 8
causal_mask = create_causal_mask(seq_len)

Q = np.random.randn(seq_len, d_model)
K = np.random.randn(seq_len, d_model)
V = np.random.randn(seq_len, d_model)

# 無遮罩（雙向）
output_bi, attn_bi = scaled_dot_product_attention(Q, K, V)

# 有因果遮罩（單向）
output_causal, attn_causal = scaled_dot_product_attention(Q, K, V, mask=causal_mask)

# 視覺化差異
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

# 因果遮罩
ax1.imshow(causal_mask, cmap='Reds', aspect='auto')
ax1.set_title('因果遮罩\n（1 = 被遮罩/不允許）')
ax1.set_xlabel('鍵位置')
ax1.set_ylabel('查詢位置')

# 雙向注意力
im2 = ax2.imshow(attn_bi, cmap='viridis', aspect='auto', vmin=0, vmax=1)
ax2.set_title('雙向注意力\n（可以看到未來）')
ax2.set_xlabel('鍵位置')
ax2.set_ylabel('查詢位置')

# 因果注意力
im3 = ax3.imshow(attn_causal, cmap='viridis', aspect='auto', vmin=0, vmax=1)
ax3.set_title('因果注意力\n（看不到未來）')
ax3.set_xlabel('鍵位置')
ax3.set_ylabel('查詢位置')

plt.colorbar(im3, ax=[ax2, ax3], label='注意力權重')
plt.tight_layout()
plt.show()

print("\n因果遮罩對以下情況至關重要：")
print("  - 自迴歸生成（GPT、語言模型）")
print("  - 防止來自未來詞元的資訊洩漏")
print("  - 每個位置只能關注自己和之前的位置")

## 關鍵要點

### 為什麼「注意力就是你所需要的」？
- **無遞迴**：並行處理整個序列
- **無卷積**：純注意力機制
- **更好的擴展性**：O(n²d) vs RNN 中的 O(n) 順序操作
- **長距離依賴**：任意位置之間的直接連接

### 核心組件：
1. **縮放點積注意力**：高效的注意力計算
2. **多頭注意力**：多個表示子空間
3. **位置編碼**：注入位置資訊
4. **前饋網路**：位置級變換
5. **層正規化**：穩定訓練
6. **殘差連接**：實現深度網路

### 架構變體：
- **編碼器-解碼器**：原始 Transformer（翻譯）
- **僅編碼器**：BERT（雙向理解）
- **僅解碼器**：GPT（自迴歸生成）

### 優點：
- 可並行化訓練（不像 RNN）
- 更好的長距離依賴
- 可解釋的注意力模式
- 在許多任務上達到最先進水平

### 影響：
- 現代 NLP 的基礎：GPT、BERT、T5 等
- 擴展到視覺：Vision Transformer（ViT）
- 多模態模型：CLIP、Flamingo
- 實現了數十億參數的大型語言模型